In [1]:
 %%time
import datetime
import shutil
import time
print(time.asctime())

import pandas as pd
import numpy as np
import autosklearn.classification
from autosklearn.classification import AutoSklearnClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
import pickle
from autosklearn.constants import *
from autosklearn import metrics
import warnings
import os
import multiprocessing
from autosklearn.metrics import make_scorer
from autosklearn.metrics import roc_auc

start = datetime.datetime.now()
print(start)


Wed Nov 14 22:00:36 2018


/usr/lib/python3.7/site-packages/pyparsing.py:2681: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )


2018-11-14 22:01:11.894262
CPU times: user 861 ms, sys: 189 ms, total: 1.05 s
Wall time: 35.8 s


In [2]:
%%time
## Constant values
data_dir = "/home/fedora/programs/hackathon/"
target_file = "orange_large_train_churn.labels"
NUMROWS = 50000
SELROWS = 25000
memory_limit       = 24576##20480  ##18432  ## 18GB RAM  
time_to_run_algo   = 86400  ##14400  4hrs ##43200  ## 12hrs
time_to_run_ensemble = 43200 ## 12 hrs  
tmp_folder = '/tmp/autosklearn_parallel_example_tmp'
output_folder = '/tmp/autosklearn_parallel_example_out'
feat_extract_txt   = "kdd2009"
dataset_name = "kdd_ds"

print(datetime.datetime.now())

2018-11-14 22:01:11.901018
CPU times: user 123 µs, sys: 0 ns, total: 123 µs
Wall time: 85.4 µs


In [3]:
## Target churn
fulltarget_df = pd.read_csv(data_dir+target_file, header=None, squeeze=True, skiprows=0, nrows=NUMROWS)


selrows = [x for x in range (0,SELROWS)]
selrows.sort() 

##traintarget_df = fulltarget_df.loc[selrows]
##print("\ntrain target shape\n", traintarget_df.shape)
#print("\ntrain target \n", traintarget_df)
traintarget_df  = pd.read_pickle("rusytrainpickle")
print("\ntrain target shape\n", traintarget_df.shape)

## target churn for testing
fulltarget_df.drop(fulltarget_df.index[selrows], inplace=True)
testtarget_df = pd.Series()
testtarget_df = fulltarget_df
print("\ntest target shape\n", testtarget_df.shape)


## Input data for training
train_df  = pd.read_pickle("rustrainpickle")
print("\nInput train data shape\n", train_df.shape)

## Fill NaN with 0's
train_df.fillna(0)

## Input data for testing
test_df  = pd.read_pickle("rustestpickle")
print("\nInput test data shape\n", test_df.shape)

## Fill NaN with 0's
test_df.fillna(0)


X_train = train_df
y_train = traintarget_df

print(datetime.datetime.now())


train target shape
 (7435, 1)

test target shape
 (25000,)

Input train data shape
 (7435, 2907)

Input test data shape
 (25000, 2907)
2018-11-14 22:01:15.220183


In [4]:
time_suffix = str(start)

# File prefixes / suffixes
for ch in [" ", ":", "-"]:
    time_suffix = time_suffix.replace(ch, "")

f_name_prefix = "automl_model_" + feat_extract_txt
f_name_suffix = "_" + time_suffix + ".txt"
temp_log_str  = "temp_log_"

f_res_name = f_name_prefix + "final_results" + f_name_suffix

##recall = make_scorer('recall', recall_score, labels = [-1,1], pos_label = 1, average = "binary")

def get_spawn_classifier(X_train, y_train):
    def spawn_classifier(seed, dataset_name):
        """Spawn a subprocess.

        auto-sklearn does not take care of spawning worker processes. This
        function, which is called several times in the main block is a new
        process which runs one instance of auto-sklearn.
        """

        # Use the initial configurations from meta-learning only in one out of
        # the four processes spawned. This prevents auto-sklearn from evaluating
        # the same configurations in four processes.
        if seed == 0:
            initial_configurations_via_metalearning = 25
            smac_scenario_args = {}
        else:
            initial_configurations_via_metalearning = 0
            smac_scenario_args = {'initial_incumbent': 'RANDOM'}

        # Arguments which are different to other runs of auto-sklearn:
        # 1. all classifiers write to the same output directory
        # 2. shared_mode is set to True, this enables sharing of data between
        # models.
        # 3. all instances of the AutoSklearnClassifier must have a different seed!
        automl = AutoSklearnClassifier(
            time_left_for_this_task=time_to_run_algo, # sec., how long should this seed fit process run
            ml_memory_limit=memory_limit, # MB, memory limit imposed on each call to a ML algorithm
            include_preprocessors=["no_preprocessing"],
            exclude_preprocessors=None,
            include_estimators = ["random_forest"],
            shared_mode=True, # tmp folder will be shared between seeds
            tmp_folder=tmp_folder,
            output_folder=output_folder,
            delete_tmp_folder_after_terminate=False,
            ensemble_size=0, # ensembles will be built when all optimization runs are finished
            initial_configurations_via_metalearning=initial_configurations_via_metalearning,
            seed=seed,
            smac_scenario_args=smac_scenario_args,
        )
        
        automl.fit(X_train, y_train, metric=metrics.roc_auc, dataset_name=dataset_name)
        
        f_name = f_name_prefix + temp_log_str + str(seed) + f_name_suffix
        
        with open (f_name, "a") as ar:
            ar.write("\n\n------------------AUTOML_SEED: " + str(seed) + "------------------------")
            ar.write ("\n\n-----------------------cv_results--------------------\n\n")
            for item in automl.cv_results_:
                ar.write (str (item) + "\n")

            ar.write ("\n\n--------------sprint_statistics-----------------------\n\n")
            ar.write (str (automl.sprint_statistics()) + "\n")
            
    print(datetime.datetime.now())    
    return spawn_classifier

In [5]:
%%time

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    for dir in [tmp_folder, output_folder]:
        try:
            shutil.rmtree(dir)
        except OSError as e:
            pass

    processes = []
    spawn_classifier = get_spawn_classifier(X_train, y_train)
    
    for i in range(7): # set this at roughly half of your cores
        p = multiprocessing.Process(target=spawn_classifier, args=(i, dataset_name))
        p.start()
        processes.append(p)
        
    for p in processes:
        p.join()

    print('Starting to build an ensemble!')
    automl = AutoSklearnClassifier(
        time_left_for_this_task=time_to_run_ensemble,
        ml_memory_limit=memory_limit,
        shared_mode=True,
        ensemble_size=50,
        ensemble_nbest=200,
        tmp_folder=tmp_folder,
        output_folder=output_folder,
        initial_configurations_via_metalearning=0,
        seed=1,
    )

    # Both the ensemble_size and ensemble_nbest parameters can be changed now if
    # necessary
    automl.fit_ensemble(
        y_train,
        task=BINARY_CLASSIFICATION,
        metric=metrics.roc_auc,
        precision='64',
        dataset_name=dataset_name
    )
    
    pickle.dump(file=open("automl1mp_model" + ".pkl", "wb"), obj=automl)
    
    with open(f_res_name, "a") as ar:
        for file in os.listdir():
            if ((f_name_prefix+temp_log_str) in file):
                with open(file, "r") as fp:
                    ar.write (fp.read())
                    ar.write("\n")
                os.remove(file)
            
        ar.write ("\n\n------------------show_models----------------------------\n\n")
        ar.write (str (automl.show_models()) + "\n")    

        ar.write ("\n\n------------------params----------------------------\n\n")
        ar.write (str (automl.get_params()) + "\n")  
    

2018-11-14 22:01:24.365028
[WARNING] [2018-11-15 00:34:13,045:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-11-15 00:34:13,045:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-11-15 01:02:59,240:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-11-15 01:02:59,240:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-11-15 05:44:56,956:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-11-15 05:44:56,956:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-11-15 07:53:00,124:smac.int

In [6]:
%%time

## Predictions
predictions = []
predictions.append(automl.predict(test_df))

## Estimated churn from test
estchurntest = np.array(predictions).sum(axis=0)
estchurntest = [-1 if item < 0 else 1 for item in estchurntest]
estchurntest = pd.Series(estchurntest)
print("\nestchurntest\n", estchurntest)



estchurntest
 0       -1
1       -1
2       -1
3       -1
4       -1
5       -1
6       -1
7       -1
8       -1
9       -1
10      -1
11      -1
12      -1
13      -1
14       1
15       1
16      -1
17      -1
18      -1
19      -1
20      -1
21      -1
22      -1
23      -1
24      -1
25      -1
26      -1
27      -1
28      -1
29      -1
        ..
24970   -1
24971   -1
24972   -1
24973   -1
24974   -1
24975   -1
24976    1
24977    1
24978    1
24979   -1
24980   -1
24981   -1
24982   -1
24983   -1
24984   -1
24985    1
24986   -1
24987   -1
24988    1
24989   -1
24990   -1
24991   -1
24992   -1
24993   -1
24994    1
24995    1
24996   -1
24997   -1
24998   -1
24999   -1
Length: 25000, dtype: int64
CPU times: user 2min 39s, sys: 18.6 s, total: 2min 58s
Wall time: 1min 45s


In [7]:
%%time
## Check accuracy
avg_vals = ["micro", "macro", "weighted", None]

with open ("automl_results.txt", "a") as ar:

    ar.write ("------------------Scores----------------------------\n\n")
    
    for avg_val in avg_vals:
    
        str_val = ("\n\nPrecision (" 
                   + str(avg_val)
                   + ") = " 
                   + str(precision_score(y_true=testtarget_df, y_pred=estchurntest, average=avg_val)))

        print (str_val)
        ar.write (str_val)
        
        str_val = ("\n\nRecall (" 
                   + str(avg_val) 
                   + ") = " 
                   + str(recall_score(y_true=testtarget_df, y_pred=estchurntest, average=avg_val)))

        print (str_val)
        ar.write (str_val)
        
        str_val = ("\n\nF1 (" 
                   + str(avg_val) 
                   + ") = " 
                   + str(f1_score(y_true=testtarget_df, y_pred=estchurntest, average=avg_val)))

        print (str_val)
        ar.write (str_val)
        
    str_val = ("\n\nconfusion_matrix"
                   + " = " 
                   + str(confusion_matrix(y_true=testtarget_df, y_pred=estchurntest)))
    print (str_val)
    ar.write (str_val)



Precision (micro) = 0.85156


Recall (micro) = 0.85156


F1 (micro) = 0.85156


Precision (macro) = 0.5695213732934564


Recall (macro) = 0.6111109457713785


F1 (macro) = 0.5813266023677893


Precision (weighted) = 0.889346943080849


Recall (weighted) = 0.85156


F1 (weighted) = 0.8685261097542553


Precision (None) = [0.94409456 0.19494819]


Recall (None) = [0.89271998 0.32950192]


F1 (None) = [0.91768881 0.24496439]


confusion_matrix = [[20687  2486]
 [ 1225   602]]
CPU times: user 274 ms, sys: 9.83 ms, total: 283 ms
Wall time: 106 ms
